In [0]:
%sql
-- L1→L2.1 transformation: Deduplication, type casting, code mapping
INSERT OVERWRITE pharma_quality.l2_1_lims.src_lims_specification
SELECT
  specification_id AS source_specification_id,
  _batch_id AS source_batch_id,
  _ingestion_timestamp AS source_ingestion_timestamp,
  _record_hash AS record_hash,
  TRIM(spec_number) AS spec_number,
  CASE WHEN spec_version RLIKE '^V?[0-9]+$' THEN CONCAT(REGEXP_EXTRACT(spec_version, '[0-9]+'), '.0') ELSE TRIM(spec_version) END AS spec_version,
  TRIM(spec_title) AS spec_title,
  CASE TRIM(UPPER(spec_type))
    WHEN 'DRUG SUBSTANCE' THEN 'DS'
    WHEN 'API' THEN 'DS'
    WHEN 'DRUG PRODUCT' THEN 'DP'
    WHEN 'FINISHED PRODUCT' THEN 'DP'
    WHEN 'RAW MATERIAL' THEN 'RM'
    WHEN 'EXCIPIENT' THEN 'EXCIP'
    WHEN 'INTERMEDIATE' THEN 'INTERMED'
    WHEN 'IN-PROCESS' THEN 'IPC'
    ELSE 'DP'
  END AS spec_type_code,
  TRIM(spec_type) AS spec_type_name,
  product_id AS product_id_lims,
  TRIM(product_name) AS product_name,
  material_id AS material_id_lims,
  TRIM(material_name) AS material_name,
  site_id AS site_id_lims,
  TRIM(site_name) AS site_name,
  market_region,
  dosage_form,
  strength,
  CASE TRIM(UPPER(status))
    WHEN 'ACTIVE' THEN 'APP'
    WHEN 'INACTIVE' THEN 'OBS'
    WHEN 'DRAFT' THEN 'DRA'
    WHEN 'OBSOLETE' THEN 'OBS'
    ELSE 'DRA'
  END AS status_code,
  TRIM(status) AS status_name,
  CAST(effective_start_date AS DATE) AS effective_start_date,
  CAST(effective_end_date AS DATE) AS effective_end_date,
  CAST(approval_date AS DATE) AS approval_date,
  TRIM(approved_by) AS approved_by,
  ctd_ref AS ctd_section,
  stage AS stage_code,
  compendia AS compendia_reference,
  superseded_by AS supersedes_spec_id,
  -- Data quality flags
  IF(effective_start_date IS NULL OR effective_end_date IS NULL OR approval_date IS NULL, TRUE, FALSE) AS dq_date_parse_error,
  IF(spec_type_code IS NOT NULL, TRUE, FALSE) AS dq_type_code_mapped,
  IF(status_code IS NOT NULL, TRUE, FALSE) AS dq_status_code_mapped,
  FALSE AS dq_duplicate_flag,
  current_timestamp() AS load_timestamp,
  TRUE AS is_current
FROM (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY specification_id ORDER BY _ingestion_timestamp DESC) AS rn
  FROM pharma_quality.l1_raw.raw_lims_specification
  WHERE _source_system = 'LIMS'
) WHERE rn = 1;

-- Repeat similar logic for pharma_quality.l2_1_lims.src_lims_spec_item and pharma_quality.l2_1_lims.src_lims_spec_limit
-- (use deduplication, type casting, and code mapping as per their DDLs)

### L1→L2.1 Transformation: Deduplication, Type Casting, Code Mapping
This step ingests raw LIMS specification data, deduplicates records, applies type casting, and maps business codes to standardized values. Data quality flags are generated for downstream validation.

**Key points:**
* Deduplication by latest ingestion timestamp
* Standardizes codes (spec_type, status, criticality, etc.)
* Generates data quality flags for date parsing, code mapping, and duplicates
* Ensures traceability with batch and record hash


| L1 Field              | L2.1 Field                | Mapping Logic / Notes                |
|-----------------------|---------------------------|--------------------------------------|
| specification_id      | source_specification_id   | Direct mapping                       |
| _batch_id             | source_batch_id           | Direct mapping                       |
| _ingestion_timestamp  | source_ingestion_timestamp| Direct mapping                       |
| _record_hash          | record_hash               | Direct mapping                       |
| spec_number           | spec_number               | Trimmed                              |
| spec_version          | spec_version              | Regex/cast, version normalization    |
| spec_title            | spec_title                | Trimmed                              |
| spec_type             | spec_type_code, spec_type_name | Code mapping, trimmed           |
| product_id            | product_id_lims           | Direct mapping                       |
| product_name          | product_name              | Trimmed                              |
| material_id           | material_id_lims          | Direct mapping                       |
| material_name         | material_name             | Trimmed                              |
| site_id               | site_id_lims              | Direct mapping                       |
| site_name             | site_name                 | Trimmed                              |
| market_region         | market_region             | Direct mapping                       |
| dosage_form           | dosage_form               | Direct mapping                       |
| strength              | strength                  | Direct mapping                       |
| status                | status_code, status_name  | Code mapping, trimmed                |
| effective_start_date  | effective_start_date      | Cast to DATE                         |
| effective_end_date    | effective_end_date        | Cast to DATE                         |
| approval_date         | approval_date             | Cast to DATE                         |
| approved_by           | approved_by               | Trimmed                              |
| ctd_ref               | ctd_section               | Direct mapping                       |
| stage                 | stage_code                | Code mapping                         |
| compendia             | compendia_reference       | Direct mapping                       |
| superseded_by         | supersedes_spec_id        | Direct mapping                       |
| dq_date_parse_error   | dq_date_parse_error       | Generated (date quality check)       |
| dq_type_code_mapped   | dq_type_code_mapped       | Generated (type code check)          |
| dq_status_code_mapped | dq_status_code_mapped     | Generated (status code check)        |
| dq_duplicate_flag     | dq_duplicate_flag         | Generated (deduplication check)      |
| load_timestamp        | load_timestamp            | Generated (current timestamp)        |
| is_current            | is_current                | Generated (TRUE)                     |
| _ingestion_id         | (not mapped)              | Not used in L2.1                     |
| _source_system        | (not mapped)              | Not used in L2.1                     |
| _source_file          | (not mapped)              | Not used in L2.1                     |
| created_date          | (not mapped)              | Not used in L2.1                     |
| modified_date         | (not mapped)              | Not used in L2.1                     |
| created_by            | (not mapped)              | Not used in L2.1                     |
| raw_payload           | (not mapped)              | Not used in L2.1                     |


### L2.2→L3 Transformation: Regulatory Acceptance Criteria & CTD Specification
This step maps L2.2 star schema records to L3 regulatory tables, propagating all relevant business fields and setting NULL/defaults for fields not present upstream. Joins to dimension tables resolve keys and enrich regulatory context.

**Key points:**
* Propagates business fields from L2.2 to L3
* Sets regulatory, product, and material fields to NULL if not available upstream
* Joins to dimension tables for context and enrichment
* Ensures traceability with source keys and load timestamp


| L2.2 Field                | L3 Field                  | Mapping Logic / Notes |
|--------------------------|--------------------------|----------------------|
| spec_key                 | spec_key                  | Direct mapping       |
| spec_number              | spec_number               | Direct mapping       |
| spec_version             | spec_version              | Direct mapping       |
| spec_title               | spec_title                | Direct mapping       |
| spec_type_code           | spec_type_code            | Direct mapping       |
| spec_type_name           | spec_type_name            | Direct mapping       |
| ctd_section              | ctd_section               | Direct mapping       |
| stage_code               | stage_code                | Direct mapping       |
| stage_name               | stage_name                | Direct mapping       |
| status_code              | status_code               | Direct mapping       |
| effective_start_date     | effective_start_date      | Direct mapping       |
| effective_end_date       | effective_end_date        | Direct mapping       |
| approval_date            | approval_date             | Direct mapping       |
| approver_name            | approver_name             | Direct mapping       |
| site_code                | site_code                 | NULL/default         |
| site_name                | site_name                 | NULL/default         |
| site_regulatory_region   | site_regulatory_region    | NULL/default         |
| market_country_code      | market_country_code       | NULL/default         |
| market_country_name      | market_country_name       | NULL/default         |
| market_status            | market_status             | NULL/default         |
| compendia_reference      | compendia_reference       | Direct mapping       |
| product_name             | product_name              | NULL/default         |
| inn_name                 | inn_name                  | NULL/default         |
| brand_name               | brand_name                | NULL/default         |
| dosage_form_code         | dosage_form_code          | NULL/default         |
| dosage_form_name         | dosage_form_name          | NULL/default         |
| route_of_administration  | route_of_administration   | NULL/default         |
| strength                 | strength                  | NULL/default         |
| nda_number               | nda_number                | NULL/default         |
| material_name            | material_name             | NULL/default         |
| material_type_code       | material_type_code        | NULL/default         |
| cas_number               | cas_number                | NULL/default         |
| molecular_formula        | molecular_formula         | NULL/default         |
| molecular_weight         | molecular_weight          | NULL/default         |
| pharmacopoeia_grade      | pharmacopoeia_grade       | NULL/default         |
| regulatory_context_code  | regulatory_context_code   | NULL/default         |
| region_code              | region_code               | NULL/default         |
| region_name              | region_name               | NULL/default         |
| regulatory_body          | regulatory_body           | NULL/default         |
| submission_type          | submission_type           | NULL/default         |
| guideline_code           | guideline_code            | NULL/default         |
| spec_item_key            | spec_item_key             | Direct mapping       |
| sequence_number          | sequence_number           | Direct mapping       |
| test_code                | test_code                 | Direct mapping       |
| test_name                | test_name                 | Direct mapping       |
| test_description         | test_description          | NULL/default         |
| test_category_code       | test_category_code        | Direct mapping       |
| test_category_name       | test_category_name        | Direct mapping       |
| test_subcategory         | test_subcategory          | Direct mapping       |
| analyte_code             | analyte_code              | Direct mapping       |
| criticality              | criticality               | Direct mapping       |
| is_required              | is_required               | Direct mapping       |
| is_compendial            | is_compendial             | Direct mapping       |
| reporting_type           | reporting_type            | Direct mapping       |
| result_precision         | result_precision          | Direct mapping       |
| result_uom_code          | result_uom_code           | NULL/default         |
| result_uom_name          | result_uom_name           | NULL/default         |
| stage_applicability      | stage_applicability       | Direct mapping       |
| test_method_number       | test_method_number        | NULL/default         |
| test_method_name         | test_method_name          | NULL/default         |
| test_method_version      | test_method_version       | NULL/default         |
| method_type_code         | method_type_code          | NULL/default         |
| analytical_technique     | analytical_technique      | NULL/default         |
| compendia_test_ref       | compendia_test_ref        | Direct mapping       |
| validation_status        | validation_status         | NULL/default         |
| ac_lower_limit           | ac_lower_limit            | Direct mapping       |
| ac_upper_limit           | ac_upper_limit            | Direct mapping       |
| ac_target                | ac_target                 | Direct mapping       |
| ac_lower_operator        | ac_lower_operator         | Direct mapping       |
| ac_upper_operator        | ac_upper_operator         | Direct mapping       |
| ac_limit_text            | ac_limit_text             | Direct mapping       |
| ac_limit_description     | ac_limit_description      | Direct mapping       |
| ac_limit_basis           | ac_limit_basis            | Direct mapping       |
| ac_uom_code              | ac_uom_code               | NULL/default         |
| ac_regulatory_basis      | ac_regulatory_basis       | NULL/default         |
| stability_time_point     | stability_time_point      | Direct mapping       |
| stability_condition      | stability_condition       | Direct mapping       |
| stability_condition_label| stability_condition_label | NULL/default         |
| is_release_spec          | is_release_spec           | NULL/default         |
| is_stability_spec        | is_stability_spec         | NULL/default         |
| nor_lower_limit          | nor_lower_limit           | NULL/default         |
| nor_upper_limit          | nor_upper_limit           | NULL/default         |
| nor_limit_description    | nor_limit_description     | NULL/default         |
| par_lower_limit          | par_lower_limit           | NULL/default         |
| par_upper_limit          | par_upper_limit           | NULL/default         |
| par_limit_description    | par_limit_description     | NULL/default         |
| is_hierarchy_valid       | is_hierarchy_valid        | NULL/default         |
| source_spec_key          | source_spec_key           | Direct mapping       |
| source_spec_item_key     | source_spec_item_key      | Direct mapping       |
| source_system_code       | source_system_code        | Direct mapping       |
| load_timestamp           | load_timestamp            | Generated            |
| data_product_version     | data_product_version      | NULL/default         |


| L1 Field              | L2.1 Field                | Mapping Logic / Notes                |
|-----------------------|---------------------------|--------------------------------------|
| spec_item_id          | source_spec_item_id        | Direct mapping                       |
| specification_id      | source_specification_id    | Direct mapping                       |
| _batch_id             | source_batch_id            | Direct mapping                       |
| _ingestion_timestamp  | source_ingestion_timestamp | Direct mapping                       |
| _record_hash          | record_hash                | Direct mapping                       |
| test_code             | test_code                  | Trimmed                              |
| test_name             | test_name                  | Trimmed                              |
| analyte_code          | analyte_code               | Trimmed                              |
| parameter_name        | parameter_name             | Trimmed                              |
| test_category         | test_category_code, test_category_name | Code mapping, trimmed   |
| test_subcategory      | test_subcategory           | Trimmed                              |
| uom                   | uom_code                   | Trimmed                              |
| criticality           | criticality_code           | Code mapping, trimmed                |
| sequence_number       | sequence_number            | Cast to INT                          |
| reporting_type        | reporting_type             | Code mapping, trimmed                |
| result_precision      | result_precision           | Cast to INT                          |
| is_required           | is_required                | Y/N/TRUE/FALSE → BOOLEAN             |
| compendia_ref         | compendia_test_ref         | Trimmed                              |
| stage_applicability   | stage_applicability        | Code mapping, trimmed                |
| test_method_id        | test_method_id_lims        | Trimmed                              |
| dq_category_mapped    | dq_category_mapped         | Generated (category mapping check)   |
| dq_criticality_mapped | dq_criticality_mapped      | Generated (criticality mapping check)|
| dq_type_cast_error    | dq_type_cast_error         | Generated (type cast check)          |
| load_timestamp        | load_timestamp             | Generated (current timestamp)        |
| is_current            | is_current                 | Generated (TRUE)                     |
| created_date          | (not mapped)               | Not used in L2.1                     |
| modified_date         | (not mapped)               | Not used in L2.1                     |
| _ingestion_id         | (not mapped)               | Not used in L2.1                     |
| _source_system        | (not mapped)               | Not used in L2.1                     |


In [0]:
%sql
-- L1→L2.1 transformation: Deduplication, type casting, code mapping for spec item
INSERT OVERWRITE pharma_quality.l2_1_lims.src_lims_spec_item (
  source_spec_item_id,
  source_specification_id,
  source_batch_id,
  source_ingestion_timestamp,
  record_hash,
  test_code,
  test_name,
  analyte_code,
  parameter_name,
  test_category_code,
  test_category_name,
  test_subcategory,
  uom_code,
  criticality_code,
  sequence_number,
  reporting_type,
  result_precision,
  is_required,
  compendia_test_ref,
  stage_applicability,
  test_method_id_lims,
  dq_category_mapped,
  dq_criticality_mapped,
  dq_type_cast_error,
  load_timestamp,
  is_current
)
SELECT
  spec_item_id AS source_spec_item_id,
  specification_id AS source_specification_id,
  _batch_id AS source_batch_id,
  _ingestion_timestamp AS source_ingestion_timestamp,
  _record_hash AS record_hash,
  TRIM(test_code) AS test_code,
  TRIM(test_name) AS test_name,
  TRIM(analyte_code) AS analyte_code,
  TRIM(parameter_name) AS parameter_name,
  CASE TRIM(UPPER(test_category))
    WHEN 'PHYSICAL' THEN 'PHY'
    WHEN 'CHEMICAL' THEN 'CHE'
    WHEN 'IMPURITY' THEN 'IMP'
    WHEN 'MICROBIOLOGICAL' THEN 'MIC'
    WHEN 'BIOLOGICAL' THEN 'BIO'
    WHEN 'STERILITY' THEN 'STER'
    WHEN 'PACKAGING' THEN 'PACK'
    ELSE NULL
  END AS test_category_code,
  TRIM(test_category) AS test_category_name,
  TRIM(test_subcategory) AS test_subcategory,
  TRIM(uom) AS uom_code,
  CASE TRIM(UPPER(criticality))
    WHEN 'CQA' THEN 'CQA'
    WHEN 'CCQA' THEN 'CCQA'
    WHEN 'NCQA' THEN 'NCQA'
    WHEN 'KQA' THEN 'KQA'
    WHEN 'REPORT' THEN 'REPORT'
    ELSE NULL
  END AS criticality_code,
  CAST(sequence_number AS INT) AS sequence_number,
  CASE TRIM(UPPER(reporting_type))
    WHEN 'NUMERIC' THEN 'NUMERIC'
    WHEN 'PASS_FAIL' THEN 'PASS_FAIL'
    WHEN 'TEXT' THEN 'TEXT'
    WHEN 'REPORT_ONLY' THEN 'REPORT_ONLY'
    ELSE NULL
  END AS reporting_type,
  CAST(result_precision AS INT) AS result_precision,
  CASE TRIM(UPPER(is_required))
    WHEN 'Y' THEN TRUE
    WHEN 'N' THEN FALSE
    WHEN 'TRUE' THEN TRUE
    WHEN 'FALSE' THEN FALSE
    ELSE NULL
  END AS is_required,
  TRIM(compendia_ref) AS compendia_test_ref,
  CASE TRIM(UPPER(stage_applicability))
    WHEN 'RELEASE' THEN 'RELEASE'
    WHEN 'STABILITY' THEN 'STABILITY'
    WHEN 'IPC' THEN 'IPC'
    WHEN 'BOTH' THEN 'BOTH'
    ELSE NULL
  END AS stage_applicability,
  TRIM(test_method_id) AS test_method_id_lims,
  IF(test_category_code IS NOT NULL, TRUE, FALSE) AS dq_category_mapped,
  IF(criticality_code IS NOT NULL, TRUE, FALSE) AS dq_criticality_mapped,
  IF(sequence_number IS NULL OR result_precision IS NULL, TRUE, FALSE) AS dq_type_cast_error,
  current_timestamp() AS load_timestamp,
  TRUE AS is_current
FROM (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY spec_item_id ORDER BY _ingestion_timestamp DESC) AS rn
  FROM pharma_quality.l1_raw.raw_lims_spec_item
  WHERE _source_system = 'LIMS'
) WHERE rn = 1;

In [0]:
%sql
-- L1→L2.1 transformation: Deduplication, type casting, code mapping for spec limit
INSERT OVERWRITE pharma_quality.l2_1_lims.src_lims_spec_limit
SELECT
  limit_id AS source_limit_id,
  spec_item_id AS source_spec_item_id,
  specification_id AS source_specification_id,
  _batch_id AS source_batch_id,
  _ingestion_timestamp AS source_ingestion_timestamp,
  _record_hash AS record_hash,
  CASE TRIM(UPPER(limit_type))
    WHEN 'ACCEPTANCE' THEN 'AC'
    WHEN 'NOR' THEN 'NOR'
    WHEN 'PROVEN ACCEPTABLE RANGE' THEN 'PAR'
    WHEN 'ALERT' THEN 'ALERT'
    WHEN 'ACTION' THEN 'ACTION'
    WHEN 'IN-PROCESS' THEN 'IPC_LIMIT'
    WHEN 'REPORT' THEN 'REPORT'
    ELSE NULL
  END AS limit_type_code,
  CAST(lower_limit AS DECIMAL(18,6)) AS lower_limit_value,
  CAST(upper_limit AS DECIMAL(18,6)) AS upper_limit_value,
  CAST(target_value AS DECIMAL(18,6)) AS target_value,
  CASE TRIM(UPPER(comparison_operator))
    WHEN 'NOT LESS THAN' THEN 'NLT'
    WHEN '>=' THEN 'NLT'
    WHEN 'GREATER THAN' THEN 'GT'
    WHEN '>' THEN 'GT'
    WHEN 'NONE' THEN 'NONE'
    ELSE NULL
  END AS lower_limit_operator,
  CASE TRIM(UPPER(comparison_operator))
    WHEN 'NOT MORE THAN' THEN 'NMT'
    WHEN '<=' THEN 'NMT'
    WHEN 'LESS THAN' THEN 'LT'
    WHEN '<' THEN 'LT'
    WHEN 'NONE' THEN 'NONE'
    ELSE NULL
  END AS upper_limit_operator,
  TRIM(limit_text) AS limit_text,
  NULL AS limit_description,
  TRIM(uom) AS uom_code,
  CASE TRIM(UPPER(limit_basis))
    WHEN 'AS_IS' THEN 'AS_IS'
    WHEN 'ANHYDROUS' THEN 'ANHYDROUS'
    WHEN 'AS_LABELED' THEN 'AS_LABELED'
    WHEN 'DRIED_BASIS' THEN 'DRIED_BASIS'
    ELSE NULL
  END AS limit_basis,
  CASE TRIM(UPPER(stage))
    WHEN 'RELEASE' THEN 'RELEASE'
    WHEN 'STABILITY' THEN 'STABILITY'
    WHEN 'IPC' THEN 'IPC'
    WHEN 'BOTH' THEN 'BOTH'
    ELSE NULL
  END AS stage_code,
  TRIM(stability_time_point) AS stability_time_point,
  TRIM(stability_condition) AS stability_condition,
  CAST(effective_start_date AS DATE) AS effective_start_date,
  CAST(effective_end_date AS DATE) AS effective_end_date,
  CASE TRIM(UPPER(calculation_method))
    WHEN '3_SIGMA' THEN '3_SIGMA'
    WHEN 'CPK' THEN 'CPK'
    WHEN 'EWMA' THEN 'EWMA'
    WHEN 'CUSUM' THEN 'CUSUM'
    WHEN 'MANUAL' THEN 'MANUAL'
    ELSE NULL
  END AS calculation_method,
  CAST(sample_size AS INT) AS sample_size,
  CAST(last_calculated_date AS DATE) AS last_calculated_date,
  CASE TRIM(UPPER(is_in_filing))
    WHEN 'Y' THEN TRUE
    WHEN 'N' THEN FALSE
    WHEN 'TRUE' THEN TRUE
    WHEN 'FALSE' THEN FALSE
    ELSE NULL
  END AS is_in_filing,
  TRIM(regulatory_basis) AS regulatory_basis,
  IF(limit_type_code IS NOT NULL, TRUE, FALSE) AS dq_limit_type_mapped,
  IF(lower_limit_operator IS NOT NULL AND upper_limit_operator IS NOT NULL, TRUE, FALSE) AS dq_operator_mapped,
  IF(lower_limit_value IS NULL OR upper_limit_value IS NULL OR target_value IS NULL, TRUE, FALSE) AS dq_numeric_cast_error,
  IF(effective_start_date IS NULL OR effective_end_date IS NULL OR last_calculated_date IS NULL, TRUE, FALSE) AS dq_date_parse_error,
  current_timestamp() AS load_timestamp,
  TRUE AS is_current
FROM (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY limit_id ORDER BY _ingestion_timestamp DESC) AS rn
  FROM pharma_quality.l1_raw.raw_lims_spec_limit
  WHERE _source_system = 'LIMS'
) WHERE rn = 1;

### L2.1→L2.2 Transformation: Star Schema with MDM Resolution
This step maps L2.1 spec limit records to the L2.2 star schema, resolving MDM keys and joining with dimension tables. All relevant business fields are propagated; fields not present in L2.1 are set to NULL/defaults. Cell-level comments clarify each mapping.

**Key points:**
* Joins to MDM dimension tables for keys (specification, item, limit type, uom)
* Propagates all business fields from L2.1
* Sets `is_conditional`, `condition_description` to NULL (not present in L2.1)
* Adds cell-level comments for maintainability
* Ensures data quality and traceability


### L2.1→L2.2 Transformation: Star Schema with MDM Resolution
This step maps L2.1 spec limit records to the L2.2 star schema, resolving MDM keys and joining with dimension tables. All relevant business fields are propagated; fields not present in L2.1 are set to NULL/defaults. Cell-level comments clarify each mapping.

**Key points:**
* Joins to MDM dimension tables for keys (specification, item, limit type, uom)
* Propagates all business fields from L2.1
* Sets `is_conditional`, `condition_description` to NULL (not present in L2.1)
* Adds cell-level comments for maintainability
* Ensures data quality and traceability


In [0]:
%sql
-- L2.1→L2.2 transformation: Star schema with MDM resolution
-- All relevant fields from L2.1 are propagated; missing fields are set to NULL/defaults
INSERT OVERWRITE pharma_quality.l2_2_spec_unified.fact_specification_limit (
  spec_key, -- MDM join
  spec_item_key, -- MDM join
  limit_type_key, -- MDM join
  uom_key, -- MDM join
  lower_limit_value, -- direct
  upper_limit_value, -- direct
  target_value, -- direct
  lower_limit_operator, -- direct
  upper_limit_operator, -- direct
  limit_text, -- direct
  limit_description, -- direct (NULL in L2.1)
  limit_basis, -- direct
  stage_code, -- direct
  stability_time_point, -- direct
  stability_condition, -- direct
  is_conditional, -- not present in L2.1, set to NULL
  condition_description, -- not present in L2.1, set to NULL
  regulatory_basis, -- direct
  is_in_filing, -- direct
  source_system_code, -- MDM join
  source_system_id, -- from source_limit_id
  effective_date, -- from effective_start_date
  effective_end_date, -- direct
  calculation_method, -- direct
  sample_size, -- direct
  last_calculated_date, -- direct
  load_timestamp, -- generated
  is_current -- generated
)
SELECT
  dim_specification.spec_key,
  dim_specification_item.spec_item_key,
  dim_limit_type.limit_type_key,
  dim_uom.uom_key,
  src_lims_spec_limit.lower_limit_value,
  src_lims_spec_limit.upper_limit_value,
  src_lims_spec_limit.target_value,
  src_lims_spec_limit.lower_limit_operator,
  src_lims_spec_limit.upper_limit_operator,
  src_lims_spec_limit.limit_text,
  src_lims_spec_limit.limit_description,
  src_lims_spec_limit.limit_basis,
  src_lims_spec_limit.stage_code,
  src_lims_spec_limit.stability_time_point,
  src_lims_spec_limit.stability_condition,
  NULL AS is_conditional, -- not present in L2.1
  NULL AS condition_description, -- not present in L2.1
  src_lims_spec_limit.regulatory_basis,
  src_lims_spec_limit.is_in_filing,
  dim_specification.source_system_code,
  src_lims_spec_limit.source_limit_id,
  src_lims_spec_limit.effective_start_date AS effective_date,
  src_lims_spec_limit.effective_end_date,
  src_lims_spec_limit.calculation_method,
  src_lims_spec_limit.sample_size,
  src_lims_spec_limit.last_calculated_date,
  current_timestamp() AS load_timestamp,
  TRUE AS is_current
FROM pharma_quality.l2_1_lims.src_lims_spec_limit
JOIN pharma_quality.l2_1_lims.src_lims_spec_item ON src_lims_spec_limit.source_spec_item_id = src_lims_spec_item.source_spec_item_id
JOIN pharma_quality.l2_1_lims.src_lims_specification ON src_lims_spec_limit.source_specification_id = src_lims_specification.source_specification_id
JOIN pharma_quality.l2_2_spec_unified.dim_specification ON src_lims_specification.source_specification_id = dim_specification.spec_id
JOIN pharma_quality.l2_2_spec_unified.dim_specification_item ON src_lims_spec_item.source_spec_item_id = dim_specification_item.spec_item_id
JOIN pharma_quality.l2_2_spec_unified.dim_limit_type ON src_lims_spec_limit.limit_type_code = dim_limit_type.limit_type_code
LEFT JOIN pharma_quality.l2_2_spec_unified.dim_uom ON src_lims_spec_limit.uom_code = dim_uom.uom_code
WHERE src_lims_spec_limit.is_current = TRUE AND src_lims_spec_item.is_current = TRUE AND src_lims_specification.is_current = TRUE;


In [0]:
%sql
-- L2.2→L3 transformation: Acceptance Criteria
INSERT OVERWRITE pharma_quality.l3_spec_products.obt_acceptance_criteria (
  spec_number,
  spec_version,
  spec_type_code,
  spec_type_name,
  stage_code,
  ctd_section,
  effective_date,
  product_name,
  inn_name,
  material_name,
  dosage_form_name,
  strength,
  nda_number,
  region_code,
  regulatory_body,
  sequence_number,
  test_name,
  test_category_code,
  test_category_name,
  test_subcategory,
  test_method_number,
  compendia_test_ref,
  result_uom_code,
  ac_lower_limit,
  ac_upper_limit,
  ac_target,
  ac_lower_operator,
  ac_upper_operator,
  ac_limit_text,
  ac_limit_description,
  ac_limit_basis,
  ac_uom_code,
  ac_stage,
  stability_time_point,
  stability_condition,
  ac_regulatory_basis,
  is_compendial,
  nor_lower_limit,
  nor_upper_limit,
  par_lower_limit,
  par_upper_limit,
  ac_width,
  nor_width,
  par_width,
  nor_tightness_pct,
  par_vs_ac_factor,
  is_hierarchy_valid,
  source_system_code,
  load_timestamp
)
SELECT
  s.spec_number,
  s.spec_version,
  s.spec_type_code,
  s.spec_type_name,
  f.stage_code,
  s.ctd_section,
  f.effective_date,
  NULL AS product_name,
  NULL AS inn_name,
  NULL AS material_name,
  NULL AS dosage_form_name,
  NULL AS strength,
  NULL AS nda_number,
  NULL AS region_code,
  NULL AS regulatory_body,
  i.sequence_number,
  i.test_name,
  i.test_category_code,
  i.test_category_name,
  i.test_subcategory,
  NULL AS test_method_number,
  i.compendia_test_ref,
  NULL AS result_uom_code,
  f.lower_limit_value AS ac_lower_limit,
  f.upper_limit_value AS ac_upper_limit,
  f.target_value AS ac_target,
  f.lower_limit_operator AS ac_lower_operator,
  f.upper_limit_operator AS ac_upper_operator,
  f.limit_text AS ac_limit_text,
  f.limit_description AS ac_limit_description,
  f.limit_basis AS ac_limit_basis,
  NULL AS ac_uom_code,
  f.stage_code AS ac_stage,
  f.stability_time_point,
  f.stability_condition,
  f.regulatory_basis AS ac_regulatory_basis,
  NULL AS is_compendial,
  NULL AS nor_lower_limit,
  NULL AS nor_upper_limit,
  NULL AS par_lower_limit,
  NULL AS par_upper_limit,
  NULL AS ac_width,
  NULL AS nor_width,
  NULL AS par_width,
  NULL AS nor_tightness_pct,
  NULL AS par_vs_ac_factor,
  NULL AS is_hierarchy_valid,
  s.source_system_code,
  f.load_timestamp
FROM pharma_quality.l2_2_spec_unified.fact_specification_limit f
JOIN pharma_quality.l2_2_spec_unified.dim_specification s ON f.spec_key = s.spec_key
JOIN pharma_quality.l2_2_spec_unified.dim_specification_item i ON f.spec_item_key = i.spec_item_key
WHERE f.is_current = TRUE AND s.is_current = TRUE AND i.is_current = TRUE
  AND f.stage_code = 'RELEASE'
  AND f.limit_type_key IN (SELECT limit_type_key FROM pharma_quality.l2_2_spec_unified.dim_limit_type WHERE limit_type_code = 'AC');

In [0]:
%sql
-- L2.2→L3 transformation: CTD-ready specification table
INSERT OVERWRITE pharma_quality.l3_spec_products.obt_specification_ctd (
  spec_key,
  spec_number,
  spec_version,
  spec_title,
  spec_type_code,
  spec_type_name,
  ctd_section,
  stage_code,
  stage_name,
  status_code,
  effective_start_date,
  effective_end_date,
  approval_date,
  approver_name,
  site_code,
  site_name,
  site_regulatory_region,
  market_country_code,
  market_country_name,
  market_status,
  compendia_reference,
  product_name,
  inn_name,
  brand_name,
  dosage_form_code,
  dosage_form_name,
  route_of_administration,
  strength,
  nda_number,
  material_name,
  material_type_code,
  cas_number,
  molecular_formula,
  molecular_weight,
  pharmacopoeia_grade,
  regulatory_context_code,
  region_code,
  region_name,
  regulatory_body,
  submission_type,
  guideline_code,
  spec_item_key,
  sequence_number,
  test_code,
  test_name,
  test_description,
  test_category_code,
  test_category_name,
  test_subcategory,
  analyte_code,
  criticality,
  is_required,
  is_compendial,
  reporting_type,
  result_precision,
  result_uom_code,
  result_uom_name,
  stage_applicability,
  test_method_number,
  test_method_name,
  test_method_version,
  method_type_code,
  analytical_technique,
  compendia_test_ref,
  validation_status,
  ac_lower_limit,
  ac_upper_limit,
  ac_target,
  ac_lower_operator,
  ac_upper_operator,
  ac_limit_text,
  ac_limit_description,
  ac_limit_basis,
  ac_uom_code,
  ac_regulatory_basis,
  stability_time_point,
  stability_condition,
  stability_condition_label,
  is_release_spec,
  is_stability_spec,
  nor_lower_limit,
  nor_upper_limit,
  nor_limit_description,
  par_lower_limit,
  par_upper_limit,
  par_limit_description,
  is_hierarchy_valid,
  source_spec_key,
  source_spec_item_key,
  source_system_code,
  load_timestamp,
  data_product_version
)
SELECT
  s.spec_key,
  s.spec_number,
  s.spec_version,
  s.spec_title,
  s.spec_type_code,
  s.spec_type_name,
  s.ctd_section,
  s.stage_code,
  s.stage_name,
  s.status_code,
  s.effective_start_date,
  s.effective_end_date,
  s.approval_date,
  s.approver_name,
  NULL AS site_code,
  NULL AS site_name,
  NULL AS site_regulatory_region,
  NULL AS market_country_code,
  NULL AS market_country_name,
  NULL AS market_status,
  s.compendia_reference,
  NULL AS product_name,
  NULL AS inn_name,
  NULL AS brand_name,
  NULL AS dosage_form_code,
  NULL AS dosage_form_name,
  NULL AS route_of_administration,
  NULL AS strength,
  NULL AS nda_number,
  NULL AS material_name,
  NULL AS material_type_code,
  NULL AS cas_number,
  NULL AS molecular_formula,
  NULL AS molecular_weight,
  NULL AS pharmacopoeia_grade,
  NULL AS regulatory_context_code,
  NULL AS region_code,
  NULL AS region_name,
  NULL AS regulatory_body,
  NULL AS submission_type,
  NULL AS guideline_code,
  i.spec_item_key,
  i.sequence_number,
  i.test_code,
  i.test_name,
  NULL AS test_description,
  i.test_category_code,
  i.test_category_name,
  i.test_subcategory,
  i.analyte_code,
  i.criticality,
  i.is_required,
  i.is_compendial,
  i.reporting_type,
  i.result_precision,
  NULL AS result_uom_code,
  NULL AS result_uom_name,
  i.stage_applicability,
  NULL AS test_method_number,
  NULL AS test_method_name,
  NULL AS test_method_version,
  NULL AS method_type_code,
  NULL AS analytical_technique,
  i.compendia_test_ref,
  NULL AS validation_status,
  f.lower_limit_value AS ac_lower_limit,
  f.upper_limit_value AS ac_upper_limit,
  f.target_value AS ac_target,
  f.lower_limit_operator AS ac_lower_operator,
  f.upper_limit_operator AS ac_upper_operator,
  f.limit_text AS ac_limit_text,
  f.limit_description AS ac_limit_description,
  f.limit_basis AS ac_limit_basis,
  NULL AS ac_uom_code,
  NULL AS ac_regulatory_basis,
  f.stability_time_point,
  f.stability_condition,
  NULL AS stability_condition_label,
  NULL AS is_release_spec,
  NULL AS is_stability_spec,
  NULL AS nor_lower_limit,
  NULL AS nor_upper_limit,
  NULL AS nor_limit_description,
  NULL AS par_lower_limit,
  NULL AS par_upper_limit,
  NULL AS par_limit_description,
  NULL AS is_hierarchy_valid,
  s.spec_key AS source_spec_key,
  i.spec_item_key AS source_spec_item_key,
  s.source_system_code,
  f.load_timestamp,
  NULL AS data_product_version
FROM pharma_quality.l2_2_spec_unified.fact_specification_limit f
JOIN pharma_quality.l2_2_spec_unified.dim_specification s ON f.spec_key = s.spec_key
JOIN pharma_quality.l2_2_spec_unified.dim_specification_item i ON f.spec_item_key = i.spec_item_key
JOIN pharma_quality.l2_2_spec_unified.dim_limit_type lt ON f.limit_type_key = lt.limit_type_key
WHERE f.is_current = TRUE AND s.is_current = TRUE AND i.is_current = TRUE
  AND f.is_in_filing = TRUE
  AND f.stage_code IN ('RELEASE', 'STABILITY');